In [1]:
def Psat(self, T):
    pop= self.getPborder(T)
    boolean=int(pop[0])

    P1=pop[1]
    P2=pop[2] 
    if boolean:
        Pmin = float(min([P1, P2])) 
        Pmax = float(max([P1, P2]))
        Tr=T/self.typeMolecule.Tc
        w=0.5*(1+scipy.tanh((10**5)*(Tr-0.6)))
        fi1=0.5*(1-scipy.tanh(8*((Tr**0.4)-1)))
        fi2=0.460*scipy.sqrt(1-(Tr-0.566)**2/(0.434**2)+0.494)

        guess = Pmin+(Pmax-Pmin)*((1-w**2)*fi1+(w**2)*fi2)  
        solution = scipy.optimize.newton(funcPsat,guess, args=(T,self))

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
import scipy.optimize
import keras.backend as K
from keras.utils import np_utils


ModuleNotFoundError: No module named 'keras'

In [22]:
data = open('corona.txt', 'r')
corona_data = [text for text in data if text.count(' ') >= 2]
vectorize = Tokenizer()
vectorize.fit_on_texts(corona_data)
corona_data = vectorize.texts_to_sequences(corona_data)
total_vocab = len(vectorize.word_index) + 1
total_length = window_size * 2

In [23]:
def cbow_model(data, window_size, total_vocab):
    context_word = []
    target = []

    for text in data:
        text_len = len(text)
        for idx, word in enumerate(text):
            begin = idx - window_size
            end = idx + window_size + 1
            if begin >= 0 and end <= text_len:
                context_word.append([text[i] for i in range(begin, end) if i != idx])
                target.append(word)

    contextual = sequence.pad_sequences(context_word, maxlen=total_length)
    final_target = np_utils.to_categorical(target, total_vocab)

    yield (contextual, final_target)


In [24]:
model = Sequential()
model.add(Embedding(input_dim=total_vocab, output_dim=100, input_length=total_length))
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
model.add(Dense(total_vocab, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [25]:
for i in range(10):
    cost = 0
    for contextual, final_target in cbow_model(corona_data, window_size, total_vocab):
        cost += model.train_on_batch(contextual, final_target)
    print(i, cost)

NameError: name 'np_utils' is not defined

In [26]:
dimensions = 100
vect_file = open('vectors.txt', 'w')
vect_file.write('{} {}\n'.format(total_vocab, dimensions))

SyntaxError: incomplete input (1877884850.py, line 3)

In [27]:
weights = model.get_weights()[0]
for text, i in vectorize.word_index.items():
    final_vec = ' '.join(map(str, list(weights[i, :])))
    vect_file.write('{} {}\n'.format(text, final_vec))

vect_file.close()

NameError: name 'vect_file' is not defined